In [1]:
from datasets import list_datasets, load_dataset
from pprint import pprint

from huggingface_hub import notebook_login
notebook_login()

In [2]:
dataset = load_dataset('sem_eval_2018_task_1', 'subtask5.spanish')

Found cached dataset sem_eval_2018_task_1 (/home/juanjo/.cache/huggingface/datasets/sem_eval_2018_task_1/subtask5.spanish/1.1.0/a7c0de8b805f1988b118882fb289ccfbbeb9085c7820b6f046b5887e234af182)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
dataset2 = load_dataset('fmplaza/EmoEvent', use_auth_token=True)

No config specified, defaulting to: emo_event/es
Found cached dataset emo_event (/home/juanjo/.cache/huggingface/datasets/fmplaza___emo_event/es/1.1.0/05d5c93b8773e3d90b52effb7132b04cd8b2ccea8d4a0750989ec3aaeb59cad4)


  0%|          | 0/3 [00:00<?, ?it/s]

In [94]:
import tweepy
import pandas as pd
import os
import unidecode
#import datetime
from datetime import timedelta, datetime 
from dateutil.relativedelta import relativedelta
import time

In [7]:
client = tweepy.Client(bearer_token = "AAAAAAAAAAAAAAAAAAAAAKKEcwEAAAAA1VgzT5upnu%2B5POQ3dbdUx0%2B5vfA%3DdslAW6ut31gZhQD9SjNKJ4scCFd8bHgbevp8eGcJprbd2LTrGf", wait_on_rate_limit = True)

In [200]:
ids_df = pd.read_csv('../Data/corpus_marcado6emociones.txt', sep=' ',skiprows=19, header=None, names = ['id','Emocion']).apply(lambda s:s.str.replace("'", ""))

In [ ]:
inicio = 0
for i in list(range(100,len(ids_df),100)) + [len(ids_df)]:
    ids_list = list(ids_df['id'][inicio:i])
    emocion_list = ids_df['Emocion'][inicio:i]
    data = client.get_tweets(ids = ids_list).data
    if data != None:
        text_list = [j.text for j in data ]
        ids_list = pd.Series([j.id for j in data ]).astype(str)
        emocion_list = ids_df[inicio:i][ids_df['id'][inicio:i].isin(ids_list)][inicio:i]['Emocion'].reset_index(drop = True)
    inicio = i
    print(i)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100


Rate limit exceeded. Sleeping for 313 seconds.


In [193]:
data != None

False

In [182]:
ids_list = list(ids_df['id'][0:100])
emocion_list = ids_df['Emocion'][0:100]
data = client.get_tweets(ids = ids_list).data
text_list = [i.text for i in data ]
ids_list = pd.Series([i.id for i in data ]).astype(str)
emocion_list = ids_df[0:100][ids_df['id'][0:100].isin(ids_list)][0:100]['Emocion'].reset_index(drop = True)

In [183]:
pd.DataFrame({'id':ids_list, 'text':text_list, 'emocion': emocion_list })

,id,text,emocion
0,465049714141200384,Buuenoos diiias ya estoy con mii pekeñaa!! Y c...,Alegria
1,468241881428951041,"Nuevos proyectos, nuevas metas, una nueva sema...",Alegria
2,469580509895729153,Hay altísimas posibilidades de a ver echo bien...,Alegria
3,469251121354391553,Me tome un taxi con el primer tachero del mund...,Alegria
4,468435168693211136,Lindos trofeos trajo el rojo ayer!!! ☺ #Feliz ...,Alegria
...,...,...,...
56,469827996774187008,Llego el día. Campus 2014 GyE.. Muchas Gracias...,Alegria
57,464611519876321281,De esas veces qe solo puedo decir..... GRACIAS...,Alegria
58,465862724283801600,"Muy lindo finde, muy jodido y lindo el festejo...",Alegria
59,469011283451604992,Estamos preparando #manualidades para éste fin...,Alegria


In [2]:
sel = pd.read_csv('../Data/SEL/SEL_full.txt', delimiter='\t', encoding= 'ISO-8859-1')

In [3]:
emociones = list(sel['Categoría'].unique())
emociones_dict ={}
for emocion in emociones:
    emociones_dict[emocion] = list(sel[sel['Categoría'] == emocion].sort_values(' PFA',  ascending=False).iloc[0:10]['Palabra'])
    for palabra in emociones_dict[emocion]:
        if not palabra.isascii():
            emociones_dict[emocion].append(unidecode.unidecode(palabra))

In [5]:
def get_tweets(emotion, words_list, number_tweets, start, end):
    emotion_words = '(' +' OR '.join(words_list) + ')'
    searched_tweets = [tweet for tweet in tweepy.Paginator(client.search_all_tweets, query = emotion_words + ' place_country:CO' ,start_time = start, end_time = end, 
                                max_results=500).flatten(limit=number_tweets)]
    tweets_df = pd.DataFrame(searched_tweets, columns = ['text'])
    tweets_df['emotion']= emotion
    tweets_df['date']= start
    return tweets_df

In [9]:
month = '04'
start_date = datetime.strptime('2021-' + month, '%Y-%m')
end_date = start_date + relativedelta(months=1)
df_list = []
while start_date < end_date:
    print(start_date)
    next_date = start_date + relativedelta(days=1)
    start_date_standard = start_date.isoformat("T") + "Z"
    next_date_standard = next_date.isoformat("T") + "Z"
    for i in emociones_dict.keys():
        print(i)
        df_list.append(get_tweets(i, emociones_dict[i], 100, start_date_standard, next_date_standard ))
        time.sleep(1)
    start_date = next_date
emociones_df = pd.concat(df_list)

2021-04-01 00:00:00
Alegría
Enojo
Miedo
Repulsión
Sorpresa
Tristeza
2021-04-02 00:00:00
Alegría
Enojo
Miedo
Repulsión
Sorpresa
Tristeza
2021-04-03 00:00:00
Alegría
Enojo
Miedo
Repulsión
Sorpresa
Tristeza
2021-04-04 00:00:00
Alegría
Enojo
Miedo
Repulsión
Sorpresa
Tristeza
2021-04-05 00:00:00
Alegría
Enojo
Miedo
Repulsión
Sorpresa
Tristeza
2021-04-06 00:00:00
Alegría
Enojo
Miedo
Repulsión
Sorpresa
Tristeza
2021-04-07 00:00:00
Alegría
Enojo
Miedo
Repulsión
Sorpresa
Tristeza
2021-04-08 00:00:00
Alegría
Enojo
Miedo
Repulsión
Sorpresa
Tristeza
2021-04-09 00:00:00
Alegría
Enojo
Miedo
Repulsión
Sorpresa
Tristeza
2021-04-10 00:00:00
Alegría
Enojo
Miedo
Repulsión
Sorpresa
Tristeza
2021-04-11 00:00:00
Alegría
Enojo
Miedo
Repulsión
Sorpresa
Tristeza
2021-04-12 00:00:00
Alegría
Enojo
Miedo
Repulsión
Sorpresa
Tristeza
2021-04-13 00:00:00
Alegría
Enojo
Miedo
Repulsión
Sorpresa
Tristeza
2021-04-14 00:00:00
Alegría
Enojo
Miedo
Repulsión
Sorpresa
Tristeza
2021-04-15 00:00:00
Alegría
Enojo
Miedo
Repulsió

In [11]:
emociones_df.to_csv('../Data/emociones_df.txt', index=False)

In [20]:
emociones_df = pd.read_csv('../Data/emociones_df.txt')

In [6]:
from transformers import BertForMaskedLM, BertTokenizer, BertForSequenceClassification

In [21]:

tokenizer = BertTokenizer.from_pretrained("/home/juanjo/BETO/pytorch/", do_lower_case=False)
model = BertForMaskedLM.from_pretrained("/home/juanjo/BETO/pytorch")
e = model.eval()

Some weights of the model checkpoint at /home/juanjo/BETO/pytorch were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
text = "[CLS] Para solucionar los [MASK] de Chile, el presidente debe [MASK] de inmediato. [SEP]"
masked_indxs = (4,11)

tokens = tokenizer.tokenize(text)
indexed_tokens = tokenizer.convert_tokens_to_ids(tokens)
tokens_tensor = torch.tensor([indexed_tokens])

predictions = model(tokens_tensor)[0]

for i,midx in enumerate(masked_indxs):
    idxs = torch.argsort(predictions[0,midx], descending=True)
    predicted_token = tokenizer.convert_ids_to_tokens(idxs[:5])
    print('MASK',i,':',predicted_token)

MASK 0 : ['problemas', 'conflictos', 'asuntos', 'males', 'temas']
MASK 1 : ['renunciar', 'actuar', 'intervenir', 'regresar', 'asumir']
